# Patronage Relationships: Production Pipeline Example
This notebook demonstrates how to use the `patronage_relationships` package in a real pipeline. Update paths and configs as needed for your environment.

In [0]:
# Import required libraries and the reusable package
from pyspark.sql import SparkSession
from patronage_relationships.core import *
from datetime import datetime

spark = SparkSession.builder.getOrCreate()

In [0]:
# Read data from production sources (update paths as needed)
psa_df = spark.read.parquet("/mnt/ci-mvi/Processed/SVeteran.SMVIPersonSiteAssociation/")
person_df = spark.read.parquet("/mnt/ci-mvi/Processed/SVeteran.SMVIPerson/")
institution_df = spark.read.parquet("/mnt/ci-mvi/Raw/NDim.MVIInstitution/")

In [0]:
# Apply reusable functions
filtered_psa = filter_psa(psa_df)
filtered_person = filter_person(person_df)
filtered_institution = filter_institution(institution_df)

joined = join_psa_institution(filtered_psa, filtered_institution)
person_institution_dups, institution_tfpi_dups = find_duplicate_relationships(joined)
clean_corr, dup_corr = remove_duplicate_relationships(joined, person_institution_dups, institution_tfpi_dups)
latest_date = get_latest_correlation_date(joined)

start = datetime.now()
lookup_table = build_correlation_lookup_table(latest_date, clean_corr, filtered_person).cache()
end = datetime.now()
print(f"Time taken: {end - start}")

json_table = build_json_correlation_table(latest_date, clean_corr, filtered_person)
json_table.createOrReplaceTempView("temp_json")

In [0]:
# Save results (repartition for parallelism if needed)
lookup_table.repartition(200, "MVIPersonICN").write.option("path", "/mnt/ci-patronage/delta_tables/correlation_lookup").mode("overwrite").saveAsTable("correlation_lookup")
# dup_corr.repartition(200, "MVIPersonICN").write.option("path", "/mnt/ci-patronage/delta_tables/duplicate_correlations").mode("overwrite").saveAsTable("duplicate_correlations")
# json_table.repartition(200, "MVIPersonICN").write.option("path", "/mnt/ci-patronage/delta_tables/CorrelationsForAllInstitutions").mode("overwrite").saveAsTable("CorrelationsForAllInstitutions")